In [1]:
import pandas as pd
import numpy as np
import os
import re

In [2]:
# Reading World Development Indicators Data
wdi_path = "WDI/data"
files = os.listdir(wdi_path)

wdi_df = pd.DataFrame()
for filename in files: # Merging data from each year
    df_aux = pd.read_csv(wdi_path + "/" + filename, error_bad_lines=False)
    df_aux['Value'] = df_aux['Value'].replace("..", np.nan)
    df_aux['Value'] = pd.to_numeric(df_aux['Value'])
    
    pivoted_df = df_aux.pivot_table(index='Country Name', 
                         columns='Series Code', 
                         values='Value').reset_index()
    
    year = re.findall('\d+|$', filename)[0]
    pivoted_df["year"] = year # Saving year in a column
    
    wdi_df = wdi_df.append(pivoted_df)


wdi_df = wdi_df.dropna(axis='columns', how='all') # Dropping empty indicators
print(wdi_df.head())

     Country Name  AG.CON.FERT.PT.ZS  AG.CON.FERT.ZS  AG.LND.AGRI.K2  \
0     Afghanistan         195.728458        4.022201        379100.0   
1         Albania                NaN      107.831247         11743.0   
2         Algeria          21.792008       23.532250        414564.0   
3  American Samoa                NaN             NaN            49.0   
4         Andorra                NaN             NaN           188.1   

   AG.LND.AGRI.ZS  AG.LND.ARBL.HA  AG.LND.ARBL.HA.PC  AG.LND.ARBL.ZS  \
0       58.067580       7765000.0           0.225638       11.893821   
1       42.857664        615100.0           0.213524       22.448905   
2       17.405923       7462100.0           0.187830        3.133044   
3       24.500000          3000.0           0.053752       15.000000   
4       40.021277           810.0           0.010383        1.723404   

   AG.LND.CREL.HA  AG.LND.CROP.ZS  ...  HD.HCI.OVRL.MA  HD.HCI.OVRL.UB  \
0       2725863.0        0.222100  ...             NaN      

In [3]:
# Reading World Happiness Report
happiness_path = "Happiness"
files = os.listdir(happiness_path)

happiness_df = pd.DataFrame()
for filename in files: # Merging data from each year
    cols_aux = ["Country", "Happiness Score"]
    df_aux = pd.read_csv(happiness_path + "/" + filename)[cols_aux]
    year = re.findall('\d+|$', filename)[0]
    df_aux["year"] = year # Saving year in a column
    
    happiness_df = happiness_df.append(df_aux)

    
selected_columns = ["Country", "Happiness Score", "year"] # Selecting only some columns
happiness_df = happiness_df[selected_columns]
happiness_df['Happiness Score'] = pd.to_numeric(happiness_df['Happiness Score'])
print(happiness_df.head())

       Country  Happiness Score  year
0  Switzerland            7.587  2015
1      Iceland            7.561  2015
2      Denmark            7.527  2015
3       Norway            7.522  2015
4       Canada            7.427  2015


In [4]:
# Merging the 2 datasets by Country and Year (LEFT JOIN)
final_df = pd.merge(wdi_df, happiness_df, left_on=["Country Name", "year"], right_on=["Country", "year"], how="left")
print(final_df.head(14))

           Country Name  AG.CON.FERT.PT.ZS  AG.CON.FERT.ZS  AG.LND.AGRI.K2  \
0           Afghanistan         195.728458        4.022201    3.791000e+05   
1               Albania                NaN      107.831247    1.174300e+04   
2               Algeria          21.792008       23.532250    4.145640e+05   
3        American Samoa                NaN             NaN    4.900000e+01   
4               Andorra                NaN             NaN    1.881000e+02   
5                Angola                NaN        8.048335    5.528731e+05   
6   Antigua and Barbuda                NaN        5.475000    9.000000e+01   
7            Arab World          17.490088       55.663125    4.785773e+06   
8             Argentina         212.515123       27.276750    1.487000e+06   
9               Armenia                NaN      145.599955    1.676700e+04   
10                Aruba                NaN             NaN    2.000000e+01   
11            Australia         253.411703       79.467859    3.

In [5]:
# Saving merged dataframe to a CSV and showing size
print(final_df.shape)

final_df.to_csv("wdi_happiness_data.csv")

(1315, 1403)
